In [39]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [40]:
"""
Bảng công thức để implement thuật toán Apriori
công thức: lift(A -> B) = confidence(A -> B) / support(B) với support(B) = count(B) / N
công thức confidence: confidence(A -> B) = support(A, B) / support(A)
công thức support: support(A, B) = count(A, B) / N với N là số lượng giao dịch, count(A, B) là số lần A và B xuất hiện cùng nhau trong giao dịch

Từ đó ta công thức của lift(A -> B) = count(A, B) / (count(A) * count(B) / N)
"""

'\nBảng công thức để implement thuật toán Apriori\ncông thức: lift(A -> B) = confidence(A -> B) / support(B) với support(B) = count(B) / N\ncông thức confidence: confidence(A -> B) = support(A, B) / support(A)\ncông thức support: support(A, B) = count(A, B) / N với N là số lượng giao dịch, count(A, B) là số lần A và B xuất hiện cùng nhau trong giao dịch\n\nTừ đó ta công thức của lift(A -> B) = count(A, B) / (count(A) * count(B) / N)\n'

In [ ]:
"""
Ví dụ cho dễ hiểu:
Giả sử có 5 giao dịch:
1. Bread, Milk
2. Bread, Butter
3. Milk, Butter
4. Bread, Milk, Butter
5. Bread, Milk, Eggs

Lk: Đây là tập hợp frequent k-itemsets (tập hợp k-mục phổ biến), nghĩa là những tập hợp k-mục mà đã thỏa mãn ngưỡng support tối thiểu (δ).
Ở ví dụ trên ta khởi tạo δ = 0.6 (60%).

Bước 1: Khởi tạo C1 và L1,

C1 chứa các mục đơn: {Bread}, {Milk}, {Butter}, {Eggs}.
Tính support cho từng mục:
Support(Bread) = 4/5 = 0.8
Support(Milk) = 4/5 = 0.8
Support(Butter) = 4/5 = 0.8
Support(Eggs) = 2/5 = 0.4
L1 giữ lại các mục có support ≥ 0.6: {Bread}, {Milk}, {Butter}.
Bước 2: Sinh C2 từ L1

C2 (tập 2-itemsets) có thể chứa các tập hợp như: {Bread, Milk}, {Bread, Butter}, {Milk, Butter}., L2 = 60% (ngưỡng support).
Tính support cho từng tập hợp:
Support({Bread, Milk}) = 3/5 = 0.6
Support({Bread, Butter}) = 3/5 = 0.6
Support({Milk, Butter}) = 3/5 = 0.6
L2 giữ lại các tập hợp có support ≥ 0.6: {Bread, Milk}, {Bread, Butter}, {Milk, Butter}.
Bước 3: Sinh C3 từ L2

C3 (tập 3-itemsets) chứa: {Bread, Milk, Butter}.
Tính support cho {Bread, Milk, Butter}:
Support({Bread, Milk, Butter}) = 2/5 = 0.4 (bị loại bỏ vì không thỏa mãn ngưỡng support 0.6).
L3 rỗng, quá trình dừng lại.

Từ kết quả trên, ta có thể tạo ra các luật kết hợp như sau:
{Bread} => {Milk} (support = 0.6, confidence = 0.75)
{Milk} => {Bread} (support = 0.6, confidence = 0.75)
{Bread} => {Butter} (support = 0.6, confidence = 0.75)
{Butter} => {Bread} (support = 0.6, confidence = 0.75)
{Milk} => {Butter} (support = 0.6, confidence = 0.75)
{Butter} => {Milk} (support = 0.6, confidence = 0.75)
{Bread, Milk} => {Butter} (support = 0.6, confidence = 1.0)
...

Và nếu như ta là người quản lý cửa hàng, ta có thể áp dụng các luật kết hợp trên để tăng doanh số bán hàng, ví dụ như:
- Khi khách hàng mua Bread, Milk thì khuyến mãi Butter
- Khi khách hàng mua Milk, Butter thì khuyến mãi Bread
- Khi khách hàng mua Bread, Butter thì khuyến mãi Milk
- ...
"""

Imply cho thuật toán

In [41]:
# Dữ liệu ví dụ - Các giao dịch giỏ hàng
data = {
    'Bread': [1, 1, 0, 1, 0, 1, 0, 0, 1, 1],
    'Butter': [1, 1, 1, 0, 1, 0, 1, 1, 0, 1],
    'Milk': [0, 1, 1, 1, 1, 1, 0, 1, 1, 0],
    'Eggs': [1, 0, 1, 1, 1, 0, 1, 1, 0, 1],
    'Cheese': [0, 1, 0, 0, 1, 1, 0, 0, 1, 1],
    'Apples': [1, 0, 1, 0, 1, 1, 0, 1, 0, 1],
    'Bananas': [0, 1, 0, 1, 0, 1, 1, 0, 1, 0]
}

In [51]:
# thử tính toán các giá trị support, confidence và lift với dữ liệu trên
def support(data, item):
    return sum(data[item]) / len(data[item])

def confidence(data, item1, item2):
    count = 0
    for i in range(len(data[item1])):
        if data[item1][i] == 1 and data[item2][i] == 1:
            count += 1
    return count / sum(data[item1])

def lift(data, item1, item2):
    return confidence(data, item1, item2) / support(data, item2)

# Tính toán các giá trị support, confidence và lift
for item1 in data.keys():
    for item2 in data.keys():
        if item1 != item2 and item1 == 'Bread':
            print(f"lift({item1} -> {item2}) = {lift(data, item1, item2)}")

lift(Bread -> Butter) = 0.7142857142857143
lift(Bread -> Milk) = 0.9523809523809524
lift(Bread -> Eggs) = 0.7142857142857143
lift(Bread -> Cheese) = 1.3333333333333333
lift(Bread -> Apples) = 0.8333333333333334
lift(Bread -> Bananas) = 1.3333333333333333


In [54]:
# So sánh với thư viện mlxtend
df = pd.DataFrame(data)
frequent_itemsets = apriori(df, min_support=0.3, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

# lấy các feature của rules
print(rules.columns)

Index(['antecedents', 'consequents', 'antecedent support',
       'consequent support', 'support', 'confidence', 'lift', 'leverage',
       'conviction', 'zhangs_metric'],
      dtype='object')


c:\Users\ADMIN\anaconda3\envs\myenv\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [55]:
# Lọc các rules có antecedents chứa 'Bread'
bread_rules = rules[rules['antecedents'].apply(lambda x: 'Bread' in x)]
print(bread_rules)

                 antecedents              consequents  antecedent support  \
0                    (Bread)                 (Cheese)                 0.6   
3                    (Bread)                (Bananas)                 0.6   
18             (Bread, Milk)                 (Cheese)                 0.4   
19           (Bread, Cheese)                   (Milk)                 0.4   
21                   (Bread)           (Cheese, Milk)                 0.6   
24          (Bread, Bananas)                   (Milk)                 0.4   
26             (Bread, Milk)                (Bananas)                 0.4   
28                   (Bread)          (Bananas, Milk)                 0.6   
30          (Bread, Bananas)                 (Cheese)                 0.4   
32           (Bread, Cheese)                (Bananas)                 0.4   
34                   (Bread)        (Bananas, Cheese)                 0.6   
60    (Bread, Bananas, Milk)                 (Cheese)                 0.4   

In [44]:
# Bước 1: Áp dụng thuật toán Apriori để tìm frequent itemsets
# Support tối thiểu: 0.5 (tức là mục phải xuất hiện trong ít nhất 50% giao dịch)
frequent_itemsets = apriori(df, min_support=0.5, use_colnames=True)

print("Frequent Itemsets:")
print(frequent_itemsets)

Frequent Itemsets:
    support                itemsets
0       0.6                 (Bread)
1       0.7                (Butter)
2       0.7                  (Milk)
3       0.7                  (Eggs)
4       0.5                (Cheese)
5       0.6                (Apples)
6       0.5               (Bananas)
7       0.6          (Eggs, Butter)
8       0.5        (Apples, Butter)
9       0.5          (Eggs, Apples)
10      0.5  (Apples, Eggs, Butter)


c:\Users\ADMIN\anaconda3\envs\myenv\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [45]:
# Bước 2: Tạo association rules từ các frequent itemsets
# Confidence tối thiểu: 0.5 (tức là rule phải đúng ít nhất 50% theo ngưỡng confidence)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

print("\nAssociation Rules:")
print(rules)


Association Rules:
         antecedents       consequents  antecedent support  \
0             (Eggs)          (Butter)                 0.7   
1           (Butter)            (Eggs)                 0.7   
2           (Apples)          (Butter)                 0.6   
3           (Butter)          (Apples)                 0.7   
4             (Eggs)          (Apples)                 0.7   
5           (Apples)            (Eggs)                 0.6   
6     (Eggs, Apples)          (Butter)                 0.5   
7   (Butter, Apples)            (Eggs)                 0.5   
8     (Eggs, Butter)          (Apples)                 0.6   
9           (Apples)    (Eggs, Butter)                 0.6   
10            (Eggs)  (Butter, Apples)                 0.7   
11          (Butter)    (Eggs, Apples)                 0.7   

    consequent support  support  confidence      lift  leverage  conviction  \
0                  0.7      0.6    0.857143  1.224490      0.11        2.10   
1              

In [46]:
rules2 = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.8)

print("\nAssociation Rules:")
print(rules2)


Association Rules:
        antecedents     consequents  antecedent support  consequent support  \
0            (Eggs)        (Butter)                 0.7                 0.7   
1          (Butter)          (Eggs)                 0.7                 0.7   
2          (Apples)        (Butter)                 0.6                 0.7   
3          (Apples)          (Eggs)                 0.6                 0.7   
4    (Eggs, Apples)        (Butter)                 0.5                 0.7   
5  (Butter, Apples)          (Eggs)                 0.5                 0.7   
6    (Eggs, Butter)        (Apples)                 0.6                 0.6   
7          (Apples)  (Eggs, Butter)                 0.6                 0.6   

   support  confidence      lift  leverage  conviction  zhangs_metric  
0      0.6    0.857143  1.224490      0.11         2.1       0.611111  
1      0.6    0.857143  1.224490      0.11         2.1       0.611111  
2      0.5    0.833333  1.190476      0.08         1

In [47]:
# lift: chỉ số lift càng lớn thì càng tốt, nếu lift > 1 thì rule này có ý nghĩa
# lift: Đo lường mức độ cải thiện của sự xuất hiện cùng nhau của hai mục so với khi chúng xuất hiện độc lập